In [ ]:
from model import BangladeshModel
import pandas as pd
import matplotlib.pyplot as plt
import time
import sys
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

In [ ]:
# params = {'seed': range(1, 3), 'scenario_probabilities': [
#     {'A': 0, 'B': 0, 'C': 0, 'D': 0},  # Scenario 0
#     {'A': 0, 'B': 0, 'C': 0, 'D': 5},  # Scenario 1
#     {'A': 0, 'B': 0, 'C': 0, 'D': 10},  # Scenario 2
#     {'A': 0, 'B': 0, 'C': 5, 'D': 10},  # Scenario 3
#     {'A': 0, 'B': 0, 'C': 10, 'D': 20},  # Scenario 4
#     {'A': 0, 'B': 5, 'C': 10, 'D': 20},  # Scenario 5
#     {'A': 0, 'B': 10, 'C': 20, 'D': 40},  # Scenario 6
#     {'A': 5, 'B': 10, 'C': 20, 'D': 40},  # Scenario 7
#     {'A': 10, 'B': 20, 'C': 40, 'D': 80},  # Scenario 8
#     # Add more scenarios as needed
# ]}
# 
# results = mesa.batch_run(
#     BangladeshModel,
#     parameters=params,
#     iterations=1,
#     max_steps=100,
#     number_processes=1,
#     data_collection_period=100000,
#     display_progress=True,
# )

In [ ]:
start = time.time()
"""
    Run simulation
    Print output at terminal
"""

# ---------------------------------------------------------------
def run_all_scenarios(seeds, run_length, scenarios):
    sys.setrecursionlimit(9999999)
    all_scenario_results = {}
    for scenario_index, scenario in enumerate(scenarios, start=0):
        print(f"Running scenario {scenario_index} with probabilities {scenario}")
        batch_results = {}
        for seed in seeds:
            print("running seed", seed)
            model = BangladeshModel(seed=seed, scenario_probabilities=scenario)
            for i in range(run_length):
                model.step()
            bridges_data = [{"unique_id": bridge.unique_id,
                             "name": bridge.name,
                             'condition': bridge.condition,
                             'length': bridge.length,
                             'total_delay_time': bridge.total_delay_time,
                             "delay_time_per_vehicle": bridge.delay_time,
                             "break_down_chance": bridge.break_down_chance,
                             "breaks_down": bridge.breaks_down} for bridge in model.bridges]
            batch_results[seed] = pd.DataFrame(bridges_data)
        all_scenario_results[f"Scenario_{scenario_index}"] = batch_results
    return all_scenario_results

# Define your scenarios
scenarios = [
    {'A': 0, 'B': 0, 'C': 0, 'D': 0},  # Scenario 0
    {'A': 0, 'B': 0, 'C': 0, 'D': 5},  # Scenario 1
    {'A': 0, 'B': 0, 'C': 0, 'D': 10},  # Scenario 2
    {'A': 0, 'B': 0, 'C': 5, 'D': 10},  # Scenario 3
    {'A': 0, 'B': 0, 'C': 10, 'D': 20},  # Scenario 4
    {'A': 0, 'B': 5, 'C': 10, 'D': 20},  # Scenario 5
    {'A': 0, 'B': 10, 'C': 20, 'D': 40},  # Scenario 6
    {'A': 5, 'B': 10, 'C': 20, 'D': 40},  # Scenario 7
    {'A': 10, 'B': 20, 'C': 40, 'D': 80},  # Scenario 8
    # Add more scenarios as needed
]

# Set your seeds and run length
seeds = range(1, 11)  # 10 seeds for each scenario
# run_length = 5 * 24 * 60  # Example run length
run_length = 7200
# Run all scenarios
all_scenario_results = run_all_scenarios(seeds, run_length, scenarios)

end = time.time()
print('run length', end - start)

In [ ]:
#all_scenario_results['Scenario_7'][7][all_scenario_results['Scenario_7'][7]['breaks_down'] == True]

In [ ]:
#all_scenario_results['Scenario_8'][7][all_scenario_results['Scenario_8'][7]['breaks_down'] == True]

In [ ]:
all_results = all_scenario_results['Scenario_1'][1].copy()
all_results = all_results[['unique_id', 'name', 'condition', 'length', 'total_delay_time',
       'delay_time_per_vehicle', 'breaks_down']] #later delay_time_per_vehicle

for scenario in all_scenario_results:
    for iteration in all_scenario_results[scenario]:
        for index, row in all_scenario_results[scenario][iteration].iterrows():
            all_results.loc[index, f'delay_time_{scenario}_{iteration}'] = row.total_delay_time
            all_results.loc[index, f'breaks_down_{scenario}_{iteration}'] = row.breaks_down
            all_results.loc[index, f'delay_car_{scenario}_{iteration}'] = row.delay_time_per_vehicle

delay_time_columns = all_results.filter(regex='^delay_time_').columns
delay_car_columns = all_results.filter(regex='^delay_car').columns

all_results['total_delay_all_scenarios'] = all_results[delay_time_columns].sum(axis=1)
all_results['avg_delay_per_car'] = all_results[delay_car_columns].mean(axis=1)

# Select columns that start with 'breaks_down_'
breaks_down_columns = all_results.filter(regex='^breaks_down_').columns

# Count how often True appears in the breaks_down columns for each row
all_results['total_breakdowns'] = all_results[breaks_down_columns].apply(lambda x: x.value_counts().get(True, 0), axis=1)


all_results[['unique_id', 'name', 'condition', 'length', 'total_delay_all_scenarios', 'total_breakdowns']]  # Show the result for the new columns

In [ ]:
delay_data = []

for i in all_scenario_results:
    for j in all_scenario_results[i]:
        delay = all_scenario_results[i][j]['total_delay_time'].sum()
        delay_data.append({
        'scenario': i,
        'seed': j,
        'total_delay_time': delay
    })

delay_data = pd.DataFrame(delay_data, columns=['scenario', 'seed', 'total_delay_time'])
delay_data 

In [ ]:
# Pivot the DataFrame so that each scenario has its own column for delay_time
pivot_df = delay_data.pivot(index='seed', columns='scenario', values='total_delay_time')

# Plotting
plt.figure(figsize=(12,6))

# Plot a line for each scenario
for scenario in pivot_df.columns:
    plt.plot(pivot_df.index, pivot_df[scenario], label=scenario)

# Labeling the axes
plt.xlabel('Seed')
plt.ylabel('Delay Time')
plt.title('Delay Time by Seed for each Scenario')

# Add a legend to the plot
plt.legend(title='Scenario')

# Show the plot
plt.show()

In [ ]:
# Assuming delay_data is already defined
pivot_df = delay_data.pivot(index='seed', columns='scenario', values='total_delay_time')

# Plotting
plt.figure(figsize=(10,5))

# Number of scenarios
num_scenarios = len(pivot_df.columns)

# Calculate the width of each bar, so they don't overlap
# There should be a small space between groups of bars, hence the 0.8 factor
bar_width = 0.8 / num_scenarios

# Iterate over each scenario to plot
for i, scenario in enumerate(pivot_df.columns):
    # Offset each bar's position to avoid overlap
    # np.arange(len(pivot_df)) gives an array of bar positions for each seed
    # i * bar_width offsets each scenario's bars
    # -0.4 to center the groups around the tick marks
    positions = np.arange(len(pivot_df)) + i * bar_width - 0.4 + bar_width/2
    
    # Plot the bars for this scenario
    plt.bar(positions, pivot_df[scenario], width=bar_width, label=scenario)

# Labeling the axes
plt.xlabel('Seed')
plt.ylabel('Delay Time')
plt.title('Delay Time by Seed for each Scenario')

# Set the x-ticks to be in the middle of the group of bars for each seed
plt.xticks(np.arange(len(pivot_df)), pivot_df.index)

plt.legend(title='Scenario', bbox_to_anchor=(1.05, 1), loc='upper left')

# Adjust the plot's layout so the legend fits outside of the plot area
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
average_delay = delay_data.groupby('scenario')['total_delay_time'].mean()
average_delay
# average_delay

In [ ]:
delay_data['total_delay_time'][delay_data['scenario'] == 'Scenario_8'].describe()

In [44]:
df = pd.read_csv('../data/merged_data.csv')
df[df['model_type'] == 'intersection'][['road', 'lon', 'lat']]

,road,lon,lat
19,N1,90.521527,23.706083
34,N1,90.546583,23.690416
243,N1,91.118166,23.478972
448,N1,91.399805,22.996389
757,N1,91.850833,22.364611
1884,N102,91.118194,23.478972
2190,N102,91.114667,24.050611
2191,N104,91.399416,23.009667
2352,N105,90.546611,23.690416
2501,N106,91.821583,22.393555


In [46]:
# The input data with roads and coordinates
road_data = """
N1 n2 90.521 23.706
N1 n104 91.381 23.01
N1 n105 90.547 23.69
N1 n102 91.118 23.479
N2 n105 90.569 23.785
N1 n106 91.833 22.369
N2 n102 91.115 24.051
N2 n204 91.347 24.148
N2 N204 91.477 24.267
N2 n207 91.51 24.294
N2 N207 91.678 24.626
N207 n208 91.766 24.471
N208 n210 91.897 24.848
N208 n2 91.875 24.878
N2 n210 91.95 24.916
"""

# Split the string into lines and then split each line into parts
lines = road_data.strip().split("\n")
split_lines = [line.split() for line in lines]

# Create a DataFrame with the appropriate column names
df_roads = pd.DataFrame(split_lines, columns=["road1", "road2", "lon", "lat"])

# Convert the lon and lat to float
df_roads["lon"] = df_roads["lon"].astype(float)
df_roads["lat"] = df_roads["lat"].astype(float)

df_roads

,road1,road2,lon,lat
0,N1,n2,90.521,23.706
1,N1,n104,91.381,23.010
2,N1,n105,90.547,23.690
3,N1,n102,91.118,23.479
4,N2,n105,90.569,23.785
5,N1,n106,91.833,22.369
6,N2,n102,91.115,24.051
7,N2,n204,91.347,24.148
8,N2,N204,91.477,24.267
9,N2,n207,91.510,24.294


In [ ]:
all_results.to_csv('../data/all_results.csv', index=False)

In [4]:
results = pd.read_csv('../data/all_results.csv')

In [6]:
results

,unique_id,name,condition,length,total_delay_time,delay_time_per_vehicle,breaks_down,delay_time_Scenario_0_1,breaks_down_Scenario_0_1,delay_car_Scenario_0_1,...,delay_car_Scenario_8_8,delay_time_Scenario_8_9,breaks_down_Scenario_8_9,delay_car_Scenario_8_9,delay_time_Scenario_8_10,breaks_down_Scenario_8_10,delay_car_Scenario_8_10,total_delay_all_scenarios,avg_delay_per_car,total_breakdowns
0,5,. id 5,A,0.0113,0,0,False,0.0,False,0.0,...,0.000000,0.000000,False,0.000000,0.0,False,0.0,0.000000,0.000000,0
1,11,. id 11,A,0.0066,0,0,False,0.0,False,0.0,...,0.000000,0.000000,False,0.000000,0.0,False,0.0,0.000000,0.000000,0
2,21,KANCHPUR PC GIRDER BRIDGE id 21,B,0.3970,0,0,False,0.0,False,0.0,...,17.048169,49277.232840,True,18.665618,0.0,False,0.0,185531.450650,0.758444,5
3,27,NOYAPARA CULVERT id 27,B,0.0063,0,0,False,0.0,False,0.0,...,12.474410,39733.558197,True,15.027821,0.0,False,0.0,229122.937537,0.936043,6
4,29,NAYABARI KASPUR BOX CULVERT id 29,A,0.0083,0,0,False,0.0,False,0.0,...,0.000000,0.000000,False,0.000000,0.0,False,0.0,132393.029721,0.536491,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377,4691,MOGLA BAZER id 4691,A,0.0208,0,0,False,0.0,False,0.0,...,10.043618,0.000000,False,0.000000,0.0,False,0.0,49150.790223,0.316981,2
1378,4697,NAIKHAI BOX CULVERT id 4697,A,0.0030,0,0,False,0.0,False,0.0,...,0.000000,0.000000,False,0.000000,0.0,False,0.0,0.000000,0.161953,1
1379,4701,JALKER KANDI id 4701,B,0.0322,0,0,False,0.0,False,0.0,...,0.000000,0.000000,False,0.000000,0.0,False,0.0,36129.061280,0.788093,5
1380,4709,SIB BARI BOX CULVERT id 4709,A,0.0030,0,0,False,0.0,False,0.0,...,0.000000,0.000000,False,0.000000,0.0,False,0.0,0.000000,0.000000,0


In [53]:
scenario_0 = pd.concat([results.iloc[:,0:7], results.iloc[:,7:37]], axis=1)
scenario_1 = pd.concat([results.iloc[:,0:7], results.iloc[:,37:67]], axis=1)
scenario_1.to_csv('scenario_1_Test')

In [51]:
# Adjusting the code snippet to follow the pattern and save each scenario to a separate CSV file

# Assuming 'df' is the DataFrame we're working with
# Initial setup: common columns count and scenario-specific columns count
common_columns_count = 7
scenario_columns_start = 7
scenario_columns_count = 30  # Number of columns for each scenario
total_scenarios = 9  # Total number of scenarios

# File paths for saved CSVs
saved_scenario_files = []

for scenario_index in range(total_scenarios):
    start_col = scenario_columns_start + scenario_index * scenario_columns_count
    end_col = start_col + scenario_columns_count
    # Creating DataFrame for each scenario
    scenario_df = pd.concat([results.iloc[:, 0:common_columns_count], results.iloc[:, start_col:end_col]], axis=1)
    # Save to CSV
    filename = f"scenario_{scenario_index}.csv"
    scenario_df.to_csv(filename, index=False)
    saved_scenario_files.append(filename)

saved_scenario_files

['scenario_0.csv',
 'scenario_1.csv',
 'scenario_2.csv',
 'scenario_3.csv',
 'scenario_4.csv',
 'scenario_5.csv',
 'scenario_6.csv',
 'scenario_7.csv',
 'scenario_8.csv']